In [1]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset, Image
from sklearn.metrics import precision_recall_fscore_support
from datasets import concatenate_datasets
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
# from PIL import Image

In [2]:
test_img = torch.stack(torch.load('test_img_tensors.pt'))

In [5]:
test_img.size()

torch.Size([1000, 1, 2048, 7, 7])

In [68]:
# importing the data
datafolder = '../../data/hateful_memes/'
train = datafolder+'train_with_features.csv'
test = datafolder+'test_with_features.csv'
dev = datafolder+'dev_with_features.csv'

# Load the train, dev, and test datasets
train_data = load_dataset('csv', data_files=train, num_proc=8).cast_column("img", Image(decode=False))
dev_data = load_dataset('csv', data_files=dev, num_proc=8).cast_column("img", Image(decode=False))
test_data = load_dataset('csv', data_files=test, num_proc=8).cast_column("img", Image(decode=False))

train_img = torch.stack(torch.load('train_img_tensors.pt'))
dev_img = torch.stack(torch.load('dev_img_tensors.pt'))
test_img = torch.stack(torch.load('test_img_tensors.pt'))

Y_train = np.asarray(train_data['train']['label'])
Y_dev = np.asarray(dev_data['train']['label'])
Y_test = np.asarray(test_data['train']['label'])

# Create datasets
train_dataset = TensorDataset(train_img, torch.Tensor(Y_train))
dev_dataset = TensorDataset(dev_img, torch.Tensor(Y_dev))
test_dataset = TensorDataset(test_img, torch.Tensor(Y_test))

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Found cached dataset csv (C:/Users/Hisha/.cache/huggingface/datasets/csv/default-d08f24387b6055f1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset csv (C:/Users/Hisha/.cache/huggingface/datasets/csv/default-ad166f4d6492fd09/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset csv (C:/Users/Hisha/.cache/huggingface/datasets/csv/default-83fc309eac32ce13/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

# CNN

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, otherwise use CPU

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(2048, 1024, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(1024, 512, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(256, 64, kernel_size=3)
        self.fc1 = nn.Linear(64 * 5 * 5, 512)
        self.fc2 = nn.Linear(512, 2)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 2048, 7, 7)  # Reshape input tensor
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = x.view(-1, 64 * 5 * 5)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Create model and optimizer
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [7]:
model.type

<bound method Module.type of CNN(
  (conv1): Conv2d(2048, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(256, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1600, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (relu): ReLU()
)>

In [63]:
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    train_correct = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        if data.shape[0] != batch_size:
            continue
        optimizer.zero_grad()
        output = model(data)
        target = target.to(torch.int64)
        # output = output.to(torch.short)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        pred = output.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.view_as(pred)).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc = 100. * train_correct / len(train_loader.dataset)
    return train_loss, train_acc

def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    test_correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            target = target.to(torch.int64)
            test_loss += criterion(output, target).item() * data.size(0)
            pred = output.argmax(dim=1, keepdim=True)
            test_correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_acc = 100. * test_correct / len(test_loader.dataset)
    return test_loss, test_acc

In [64]:
# Define hyperparameters
num_epochs = 20
criterion = nn.CrossEntropyLoss()

# Train the model
for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    dev_loss, dev_acc = test(model, dev_loader, criterion, device)
    print(f'Epoch {epoch}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Dev Loss: {dev_loss:.4f}, Dev Acc: {dev_acc:.2f}%')

Epoch 1: Train Loss: 0.6448, Train Acc: 64.19%, Dev Loss: 0.6916, Dev Acc: 50.60%
Epoch 2: Train Loss: 0.6239, Train Acc: 64.69%, Dev Loss: 0.6900, Dev Acc: 51.00%
Epoch 3: Train Loss: 0.5761, Train Acc: 68.69%, Dev Loss: 1.0591, Dev Acc: 49.20%
Epoch 4: Train Loss: 0.5529, Train Acc: 69.31%, Dev Loss: 0.7299, Dev Acc: 50.60%
Epoch 5: Train Loss: 0.4837, Train Acc: 74.44%, Dev Loss: 0.7245, Dev Acc: 49.60%
Epoch 6: Train Loss: 0.4542, Train Acc: 76.28%, Dev Loss: 0.7690, Dev Acc: 50.80%
Epoch 7: Train Loss: 0.4213, Train Acc: 79.16%, Dev Loss: 0.8157, Dev Acc: 52.80%
Epoch 8: Train Loss: 0.3797, Train Acc: 81.94%, Dev Loss: 1.5422, Dev Acc: 50.80%
Epoch 9: Train Loss: 0.3490, Train Acc: 83.67%, Dev Loss: 2.2106, Dev Acc: 50.40%
Epoch 10: Train Loss: 0.3651, Train Acc: 83.35%, Dev Loss: 1.5740, Dev Acc: 50.20%
Epoch 11: Train Loss: 0.3237, Train Acc: 85.75%, Dev Loss: 1.3484, Dev Acc: 49.20%
Epoch 12: Train Loss: 0.3060, Train Acc: 86.48%, Dev Loss: 1.7573, Dev Acc: 49.40%
Epoch 13: Tra

In [65]:
def predict(model, test_loader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            predictions.extend(pred.cpu().numpy().tolist())

    return [p[0] for p in predictions]

# Use the predict function to predict the labels of test data
predictions_test = predict(model, test_loader, device)

results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_test, predictions_test, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,0.50212,0.502121,0.501992


In [66]:
# Use the predict function to predict the labels of dev data
predictions_dev = predict(model, dev_loader, device)

results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_dev, predictions_dev, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,0.476319,0.476453,0.475463


In [69]:
# Use the predict function to predict the labels of dev data
predictions_train = predict(model, train_loader, device)

results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_train, predictions_train, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,0.793168,0.818637,0.79494


In [70]:
df_train = pd.read_csv(train, keep_default_na=False)
df_dev = pd.read_csv(dev, keep_default_na=False)
df_test = pd.read_csv(test, keep_default_na=False)

df_train['ResNet_nn'] = predictions_train
df_dev['ResNet_nn'] = predictions_dev
df_test['ResNet_nn'] = predictions_test

df_train.to_csv(datafolder+'train_with_features.csv', index=False)
df_dev.to_csv(datafolder+'dev_with_features.csv', index=False)
df_test.to_csv(datafolder+'test_with_features.csv', index=False)

# SVM: Default params

In [71]:
train_X = [np.array(x.cpu()).flatten() for x in train_img]
dev_X = [np.array(x.cpu()).flatten() for x in dev_img]
test_X = [np.array(x.cpu()).flatten() for x in test_img]

In [8]:
from sklearn.svm import SVC
clf_svc = SVC(kernel='rbf') # parameter C was selected based on grid search
clf_svc.fit(train_X, Y_train)
Y_pred = clf_svc.predict(dev_X)
results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_dev, Y_pred, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,0.518329,0.502408,0.366971


In [9]:
Y_pred2 = clf_svc.predict(test_X)

In [10]:
df_dev = pd.read_csv(dev, keep_default_na=False)
df_test = pd.read_csv(test, keep_default_na=False)

df_dev['ResNet_svm_rbf_kernel'] = Y_pred
df_test['ResNet_svm_rbf_kernel'] = Y_pred2

df_dev.to_csv(datafolder+'dev_with_features.csv', index=False)
df_test.to_csv(datafolder+'test_with_features.csv', index=False)

# SVM: C10

In [11]:
from sklearn.svm import SVC
clf_svc = SVC(kernel='rbf', C=10) # parameter C was selected based on grid search
clf_svc.fit(train_X, Y_train)
Y_pred = clf_svc.predict(dev_X)
results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_dev, Y_pred, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,0.503555,0.501872,0.437037


In [12]:
Y_pred2 = clf_svc.predict(test_X)

In [13]:
# df_dev = pd.read_csv(dev, keep_default_na=False)
# df_test = pd.read_csv(test, keep_default_na=False)

df_dev['ResNet_svm_rbf_kernelC10'] = Y_pred
df_test['ResNet_svm_rbf_kernelC10'] = Y_pred2

df_dev.to_csv(datafolder+'dev_with_features.csv', index=False)
df_test.to_csv(datafolder+'test_with_features.csv', index=False)

# SVM: Linear Kernel

In [14]:
from sklearn.svm import SVC
clf_svc = SVC(kernel='linear') # parameter C was selected based on grid search
clf_svc.fit(train_X, Y_train)
Y_pred = clf_svc.predict(dev_X)
results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_dev, Y_pred, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,0.506658,0.505529,0.484847


In [15]:
Y_pred2 = clf_svc.predict(test_X)

In [16]:
# df_dev = pd.read_csv(dev, keep_default_na=False)
# df_test = pd.read_csv(test, keep_default_na=False)

df_dev['ResNet_svm_linear_kernel'] = Y_pred
df_test['ResNet_svm_linear_kernel'] = Y_pred2

df_dev.to_csv(datafolder+'dev_with_features.csv', index=False)
df_test.to_csv(datafolder+'test_with_features.csv', index=False)

# SVM: Linear and C10

In [72]:
from sklearn.svm import SVC
clf_svc = SVC(kernel='linear', C=10) # parameter C was selected based on grid search
clf_svc.fit(train_X, Y_train)
Y_pred = clf_svc.predict(dev_X)
results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_dev, Y_pred, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,0.504545,0.504081,0.492002


In [73]:
# df_dev = pd.read_csv(dev, keep_default_na=False)
# df_test = pd.read_csv(test, keep_default_na=False)
Y_pred2 = clf_svc.predict(test_X)
Y_pred3 = clf_svc.predict(train_X)

df_train['ResNet_svm_linear_kernelC10'] = Y_pred3
df_dev['ResNet_svm_linear_kernelC10'] = Y_pred
df_test['ResNet_svm_linear_kernelC10'] = Y_pred2

df_train.to_csv(datafolder+'train_with_features.csv', index=False)
df_dev.to_csv(datafolder+'dev_with_features.csv', index=False)
df_test.to_csv(datafolder+'test_with_features.csv', index=False)

In [74]:
results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_test, Y_pred2, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,0.492996,0.493737,0.481481


In [75]:
results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_train, Y_pred3, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

,precision,recall,F1
0,1.0,1.0,1.0
